<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/CIFAR_10N_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning repository of CIFAR-10H Annotation
Paper: Human uncertainty makes classification more robust (https://arxiv.org/pdf/1908.07086.pdf)

Label:  CIFAR10 [0: airplane, 1: automobile, 2: bird, 3: cat, 4: deer, 5: dog, 6: frog, 7: horse, 8: ship, 9: truck] <br>

<img src="https://miro.medium.com/max/1010/1*r8S5tF_6naagKOnlIcGXoQ.png" alt="alternatetext">




In [1]:
!git clone https://github.com/UCSC-REAL/cifar-10-100n.git
%cd cifar-10-100n

Cloning into 'cifar-10-100n'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 109 (delta 47), reused 70 (delta 20), pack-reused 0
Receiving objects: 100% (109/109), 869.83 KiB | 3.02 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/cifar-10-100n


# main script

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=1024, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=2048, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=10, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets, ad) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

# CIFAR-10H dataloader

In [3]:
from PIL import Image
import numpy as np
import torchvision

class CIFAR10N(torchvision.datasets.CIFAR10):

    def __init__(self, root,  rand_number=0, train=False, transform=None, target_transform=None,
                 download=False, istrain=False):
        super(CIFAR10N, self).__init__(root, train, transform, target_transform, download)
        self.istrain = istrain 
        self.transform = transform
        self.target_transform = target_transform
        ann_all = torch.load('./data/CIFAR-10_human.pt')
        ann_ = np.array([ann_all['random_label1'], ann_all['random_label2'], ann_all['random_label3']]).transpose(1,0)
        self.ad = np.zeros((50000, 10))
        for idx, ann_per_img in enumerate(ann_):
            for ann_per_rater in ann_per_img:
                self.ad[idx, ann_per_rater] += 1

    def __getitem__(self, index: int):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        if self.istrain:
            ad = self.ad[index]
            return img, target, ad
        else:
            return img, target


# Run script

In [4]:
seed_everything()
mean_cifar10, std_cifar10 = (0.5071, 0.4866, 0.4409), (0.2009, 0.1984, 0.2023)
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(), transforms.ToTensor(),
            transforms.Normalize(mean_cifar10, std_cifar10), ])
transform_test = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10),])

train_dataset = CIFAR10N(root='./data', train=True, download=True, transform=transform_train, istrain=True)
test_dataset = CIFAR10N(root='./data', train=False, download=True, transform=transform_test, istrain=False)
#test_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
print('train samples:',len(train_dataset), 'test samples:',len(test_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

model = models.resnet34(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, args.num_classes)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

best_epoch, best_acc = 0.0, 0
for epoch in range(args.num_epoch):
    train(model, train_loader, criterion, optimizer)
    accuracy = test(model, test_loader)
    if accuracy > best_acc:
        patience = 0
        best_acc = accuracy
        best_epoch = epoch
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), 'best_model_cifar10h.pth.tar')
    print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
            epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
train samples: 50000 test samples: 10000


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch: 0  acc: 0.3324  best epoch: 0  best acc: 0.3324
epoch: 1  acc: 0.3735  best epoch: 1  best acc: 0.3735
epoch: 2  acc: 0.6147  best epoch: 2  best acc: 0.6147
epoch: 3  acc: 0.4401  best epoch: 2  best acc: 0.6147
epoch: 4  acc: 0.5512  best epoch: 2  best acc: 0.6147
epoch: 5  acc: 0.5408  best epoch: 2  best acc: 0.6147
epoch: 6  acc: 0.5310  best epoch: 2  best acc: 0.6147
epoch: 7  acc: 0.2718  best epoch: 2  best acc: 0.6147
epoch: 8  acc: 0.3890  best epoch: 2  best acc: 0.6147
epoch: 9  acc: 0.4624  best epoch: 2  best acc: 0.6147
epoch: 10  acc: 0.4945  best epoch: 2  best acc: 0.6147
epoch: 11  acc: 0.5159  best epoch: 2  best acc: 0.6147
epoch: 12  acc: 0.5573  best epoch: 2  best acc: 0.6147
epoch: 13  acc: 0.5759  best epoch: 2  best acc: 0.6147
epoch: 14  acc: 0.6089  best epoch: 2  best acc: 0.6147
epoch: 15  acc: 0.6188  best epoch: 15  best acc: 0.6188
epoch: 16  acc: 0.6477  best epoch: 16  best acc: 0.6477
epoch: 17  acc: 0.6513  best epoch: 17  best acc: 0.6513

In [5]:
import tensorflow_probability as tfp

def evaluation_all(model, testloader):
    model.eval()
    logits_list = []
    labels_list = []
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            logits_list.append(outputs)
            labels_list.append(targets)

        logits = torch.cat(logits_list).cpu().numpy()
        labels = torch.cat(labels_list).cpu().numpy()
    return correct / total, logits, labels

model.load_state_dict(torch.load('best_model_cifar10h.pth.tar'))
acc, logits_tf, labels_tf = evaluation_all(model, test_loader)
ece = tfp.stats.expected_calibration_error(args.num_classes, logits=logits_tf, labels_true=labels_tf, labels_predicted=np.argmax(logits_tf,1))
print("Acc:{:.4f}, ECE:{:.4f}".format(acc, np.array(ece)))

Acc:0.8076, ECE:0.0799
